#Jupyter notebook to train our model



##Step 1: Installing libraries

In [0]:
!git clone https://github.com/Mattveloso/Mask_RCNN.git

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#!git clone https://github.com/philferriere/cocoapi.git

In [0]:
!pwd

In [0]:
cd Mask_RCNN

In [0]:
!python3 setup.py install

In [0]:
!pip install tensorflow==1.15.0
!pip install gast==0.3.2
!pip install keras==2.2.5
!pip install -r requirements.txt
!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI

##Step 2: Importing Libraries and Running



In [0]:
# File for bicycle type Classification Main source is MachineLearningMastery.com
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

#from pycocotools import *
from samples.coco import coco
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn import visualize

from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

%matplotlib inline

ROOT_DIR = os.getcwd()
g_path = '/content/' #general path for files

#directory to save logs
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
#get traned weights from google drive
COCO_MODEL_PATH = '/content/drive/My Drive/Colab/mask_rcnn_coco.h5'
#download weights if needed
if not os.path.exists(COCO_MODEL_PATH):
  utils.download_trained_weights(COCO_MODEL_PATH)

#Directory of images to run detection on
IMAGE_DIR = g_path+'Mask_RCNN/bikes/images/'

###Config and Class names

In [0]:
class InferenceConfig(coco.CocoConfig):
  #set batch size to 1 since we'lll be running inference on one image at a time. Batch size = GPU_COUNT*IMAGES_PER_GPU
  GPU_COUNT=1
  IMAGES_PER_GPU=1 #i want to change to 1
  NUM_CLASSES=5

config = InferenceConfig()
config.display()

In [0]:
class_names=['BG', 'RoadBike', 'KidsBike','MountainBike','UrbanBike']

###MachineLearningMastery based code below

In [0]:
# %% class that defines and loads the Roadbike dataset
class BikesDataset(Dataset):
	# load the dataset definitions
	def load_dataset(self, dataset_dir, is_train=True):
		# define classes
		self.add_class('dataset', 1, "RoadBike")
		self.add_class('dataset', 2, "KidsBike")
		self.add_class('dataset', 3, "MountainBike")
		self.add_class('dataset', 4, "UrbanBike")
		# define data locations
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annots/'
		# find all images
		
		for filename in RoadBikes_image_list:
			# extract image id
			image_id = filename[:-4]
			# skip all images after 150 if we are building the train set
			if is_train and float(image_id) >= 125:
				continue
			# skip all images before 150 if we are building the test/val set
			if not is_train and float(image_id) < 125:
				continue
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			# add to dataset
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path,num_ids=1)

		for filename in KidsBikes_image_list:
			# extract image id
			image_id = filename[:-4]
			# skip all images after 150 if we are building the train set
			if is_train and float(image_id) >= 250:
				continue
			# skip all images before 150 if we are building the test/val set
			if not is_train and float(image_id) < 250:
				continue
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			# add to dataset
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path,num_ids=2)

		#mountain bikes go from 97 to 144 , list= MTB_image_list
		for filename in MTB_image_list:
			# extract image id
			image_id = filename[:-4]
			# skip all images after 150 if we are building the train set
			if is_train and float(image_id) >= 400:
				continue
			# skip all images before 150 if we are building the test/val set
			if not is_train and float(image_id) < 400:
				continue
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			# add to dataset
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path,num_ids=3)
	 
		#Urban bikes go from 145 to 173, list = UrbanBikes_image_list
		for filename in UrbanBikes_image_list:
			# extract image id
			image_id = filename[:-4]
			# skip all images after 150 if we are building the train set
			if is_train and float(image_id) >= 510:
				continue
			# skip all images before 150 if we are building the test/val set
			if not is_train and float(image_id) < 510:
				continue
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			# add to dataset
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path,num_ids=4)

	# extract bounding boxes from an annotation file
	def extract_boxes(self, filename):
		# load and parse the file
		tree = ElementTree.parse(filename)
		# get the root of the document
		root = tree.getroot()
		# extract each bounding box
		boxes = list()
		for box in root.findall('.//bndbox'):
			xmin = int(box.find('xmin').text)
			ymin = int(box.find('ymin').text)
			xmax = int(box.find('xmax').text)
			ymax = int(box.find('ymax').text)
			coors = [xmin, ymin, xmax, ymax]
			boxes.append(coors)
		# extract image dimensions
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height, filename

	# load the masks for an image
	def load_mask(self, image_id):
		# get details of image
		info = self.image_info[image_id]
		# define box file location
		path = info['annotation']
		# load XML
		boxes, w, h, filename = self.extract_boxes(path)
		# create one array for all masks, each on a different channel
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			masks[row_s:row_e, col_s:col_e, i] = 1
			ID = filename.split('/')
			if int(ID[-1][:-4]) <= 160:
				class_ids.append(self.class_names.index('RoadBike'))
			elif (int(ID[-1][:-4]) > 160) and  (int(ID[-1][:-4]) < 279) :
				class_ids.append(self.class_names.index('KidsBike'))
			elif (int(ID[-1][:-4]) >= 279) and  (int(ID[-1][:-4]) < 439) :
				class_ids.append(self.class_names.index('MountainBike'))
			elif (int(ID[-1][:-4]) >= 439) and  (int(ID[-1][:-4]) <= 524) :
				class_ids.append(self.class_names.index('UrbanBike'))		
		return masks, asarray(class_ids, dtype='int32')

	# load an image reference
	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

# define a configuration for the model
class BikesConfig(Config):
	# define the name of the configuration
	NAME = "Bikes_Cfg"
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1
	# number of classes (background + bikes) 
	NUM_CLASSES = 5
	# number of training steps per epoch
	STEPS_PER_EPOCH = 132

In [0]:
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import Dataset
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image

# define the prediction configuration
class PredictionConfig(Config):
	# define the name of the configuration
	NAME = "Bikes_Cfg"
	# number of classes (background + kangaroo)
	NUM_CLASSES = 1 + 4
	# simplify GPU config
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

# plot a number of photos with ground truth and predictions
def plot_actual_vs_predicted(dataset, model, cfg, n_images=20):
	# load image and mask
	for i in range(n_images):
		# load the image and mask
		image = dataset.load_image(i)
		mask, _ = dataset.load_mask(i)
		#classe = dataset.load_class(i)
		#print(classe)
		# convert pixel values (e.g. center)
		scaled_image = mold_image(image, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)[0]
		# define subplot
		pyplot.subplot(n_images, 2, i*2+1)
		# plot raw pixel data
		pyplot.imshow(image)
		pyplot.title('Actual')
		# plot masks
		for j in range(mask.shape[2]):
			pyplot.imshow(mask[:, :, j], cmap='gray', alpha=0.3)
		# get the context for drawing boxes
		pyplot.subplot(n_images, 2, i*2+2)
		# plot raw pixel data
		pyplot.imshow(image)
		pyplot.title('Predicted')
		ax = pyplot.gca()
		# plot each box
		for box in yhat['rois']:
			# get coordinates
			y1, x1, y2, x2 = box
			# calculate width and height of the box
			width, height = x2 - x1, y2 - y1
			# create the shape
			rect = Rectangle((x1, y1), width, height, fill=False, color='red')
			# draw the box
			ax.add_patch(rect)
	# show the figure
	pyplot.show()

# calculate the mAP for a model on a given dataset
def evaluate_model(dataset, model, cfg):
	APs = list()
	for image_id in dataset.image_ids:
		# load image, bounding boxes and masks for the image id
		image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
		# convert pixel values (e.g. center)
		scaled_image = mold_image(image, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)
		# extract results for first sample
		r = yhat[0]
		# calculate statistics, including AP
		AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
		print('Real: '+ gt_class_id +' | Predicted: '+ r['class_ids'])
		# store
		APs.append(AP)
	# calculate the mean AP across all images
	mAP = mean(APs)
	return mAP

##Step 3: Train Neural Network on Data


In [0]:
RoadBikes_image_list=[] #creates a list with image name for Road Bikes
for element in list(range(1,161)):
  RoadBikes_image_list.append(str(element)+'.jpg')

KidsBikes_image_list=[] #creates a list with image name for Kids Bikes
for element in list(range(161,279)):
  KidsBikes_image_list.append(str(element)+'.jpg')

MTB_image_list=[] 
for element in list(range(279,439)):
  MTB_image_list.append(str(element)+'.jpg')

UrbanBikes_image_list=[] 
for element in list(range(439,525)):
  UrbanBikes_image_list.append(str(element)+'.jpg')

In [0]:
# %% prepare train set (make sure to import the model before doing so)
train_set = BikesDataset()
train_set.load_dataset(g_path+'Mask_RCNN/bikes/', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# %% prepare train set (make sure to import the model before doing so)

# prepare test/val set
test_set = BikesDataset()
test_set.load_dataset(g_path+'Mask_RCNN/bikes/', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
# prepare config

config = BikesConfig()
config.display()
# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)
# load weights (mscoco) and exclude the output layers
model.load_weights('/content/drive/My Drive/Colab/mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
# train weights (output layers or 'heads')
# model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=5, layers='heads')
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=50, layers='heads') #training for longer to improve training set performance

##Step 4: Test on individual examples

In [0]:
# Create model object in inference mode
# model = modellib.MaskRCNN(mode='inference', model_dir=MODEL_DIR, config=config)
model = modellib.MaskRCNN(mode='inference', model_dir=MODEL_DIR, config=PredictionConfig())


#load weights
# model.load_weights(COCO_MODEL_PATH, by_name=True)
# if choosing another model, make sure configurations match (especially number of classes)
Model_Path ='/content/Mask_RCNN/bikes_cfg20200528T1628/mask_rcnn_bikes_cfg_0048.h5' #add model created here.
model.load_weights(Model_Path, by_name=True)

Re-starting from epoch 48


###With training done, we proceed to test

In [0]:
# Load a random image from the images folder
file_names = next(os.walk(IMAGE_DIR))[2]
# image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)),plugin='matplotlib')
image_list = RoadBikes_image_list+KidsBikes_image_list+MTB_image_list+UrbanBikes_image_list
new_list =[]
for image in image_list:
  image = skimage.io.imread(os.path.join(IMAGE_DIR, image),plugin='matplotlib')
  new_list.append(image)


bike = 134
for bike in list(range(400,440)):
    
  # Run detection
  # 
  results = model.detect([new_list[bike]], verbose=1)

  # Visualize results 0
  r = results[0]
  visualize.display_instances(new_list[bike], r['rois'], r['masks'], r['class_ids'], 
                              class_names, r['scores'])

  #make sure to create check to see if the model detected something worth printing, and avoid traceback!
  id = r['class_ids']

  if not id.any():
    print('predicted no object in image')
  else:
    print(class_names[id[0]])

###Code for further testing (to be added if necessary)

In [0]:
# # load the train dataset
# train_set = BikesDataset()
# train_set.load_dataset('/content/Mask_RCNN/bikes', is_train=True)
# train_set.prepare()
# print('Train: %d' % len(train_set.image_ids))
# # load the test dataset
# test_set = BikesDataset()
# test_set.load_dataset('/content/Mask_RCNN/bikes', is_train=False)
# test_set.prepare()
# print('Test: %d' % len(test_set.image_ids))
# # create config
# cfg = PredictionConfig()
# # define the model kangaroo
# model = MaskRCNN(mode='inference', model_dir='./', config=cfg)
# # load model weights
# model_path = '/content/Mask_RCNN/bikes_cfg20200521T1953/mask_rcnn_bikes_cfg_0005.h5' # Change
# model.load_weights(model_path, by_name=True)
# # plot predictions for train dataset
# plot_actual_vs_predicted(train_set, model, cfg)
# # plot predictions for test dataset
# plot_actual_vs_predicted(test_set, model, cfg)

In [0]:
# # load the train dataset
# train_set = BikesDataset()
# train_set.load_dataset('/content/Mask_RCNN/bikes', is_train=True)
# train_set.prepare()
# print('Train: %d' % len(train_set.image_ids))
# # load the test dataset
# test_set = BikesDataset()
# test_set.load_dataset('/content/Mask_RCNN/bikes', is_train=False)
# test_set.prepare()
# print('Test: %d' % len(test_set.image_ids))
# # create config
# cfg = PredictionConfig()
# # define the model
# model = MaskRCNN(mode='inference', model_dir='./', config=cfg)
# # load model weights
# model.load_weights('/content/Mask_RCNN/bikes_cfg20200521T1953/mask_rcnn_bikes_cfg_0005.h5', by_name=True)
# # evaluate model on training dataset
# train_mAP = evaluate_model(train_set, model, cfg)
# print("Train mAP: %.3f" % train_mAP)
# # evaluate model on test dataset
# test_mAP = evaluate_model(test_set, model, cfg)
# print("Test mAP: %.3f" % test_mAP)